<a href="https://colab.research.google.com/github/JaveriaAmjad1/Langchain-/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain-pinecone
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hi")



[0.0014760760823264718,
 -0.029935583472251892,
 -0.01783810183405876,
 -0.047277648001909256,
 0.027684327214956284,
 -0.00889522209763527,
 0.030663948506116867,
 -0.01426528301090002,
 0.003124301554635167,
 0.021755851805210114,
 0.056663304567337036,
 0.018048927187919617,
 -0.006203501485288143,
 -0.05175986513495445,
 -0.020865578204393387,
 0.006319219246506691,
 0.004397958982735872,
 0.00570949399843812,
 0.010415352880954742,
 -0.02124374359846115,
 0.04982486367225647,
 0.03187477961182594,
 -0.010323568247258663,
 -0.005915531888604164,
 0.013109584338963032,
 -0.04946610704064369,
 0.0284867063164711,
 -0.04388665407896042,
 -0.028827698901295662,
 0.06491994857788086,
 -0.08532701432704926,
 0.03090032935142517,
 -0.053116269409656525,
 -0.003776727942749858,
 0.05218495428562164,
 -0.08170139044523239,
 0.023761693388223648,
 0.028207849711179733,
 -0.013105872087180614,
 0.046341318637132645,
 0.00500134751200676,
 -0.0534532368183136,
 -0.05413735657930374,
 -0.033191

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("documents.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)



[Document(metadata={'source': 'documents.txt'}, page_content="Muhammad Ali Jinnah was a barrister, politician, and the founder of Pakistan. Jinnah served as the leader of the All-India Muslim League from 1913 until the inception of Pakistan on 14 August 1947 and then as Pakistan's first governor-general until his death."), Document(metadata={'source': 'documents.txt'}, page_content="Born at Wazir Mansion in Karachi, Jinnah was trained as a barrister at Lincoln's Inn in London, England. Upon his return to India, he enrolled at the Bombay High Court, and took an interest in national politics, which eventually replaced his legal practice. Jinnah rose to prominence in the Indian National Congress in the first two decades of the 20th century. In these early years of his political career, Jinnah advocated Hindu–Muslim unity, helping to shape the 1916 Lucknow Pact between the"), Document(metadata={'source': 'documents.txt'}, page_content='to shape the 1916 Lucknow Pact between the Congress an

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
from google.colab import userdata

index_name = "quickstart4"
API_KEY = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=API_KEY)

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index)

!pip install tqdm



In [ ]:
from uuid import uuid4
from tqdm import tqdm

vectors = []
for doc in tqdm(docs, desc="Uploading embeddings to Pinecone"):
    # Generate a unique ID using uuid4
    doc_id = str(uuid4())  # Generate a unique ID

    # Create the embedding for the document
    vector = embeddings.embed_query(doc.page_content)  # Embeds the document content

    # Metadata for the document
    metadata = {"text": doc.page_content}

    # Add the document to Pinecone using index.upsert()
    index.upsert([(doc_id, vector, metadata)])


Uploading embeddings to Pinecone: 100%|██████████| 29/29 [00:09<00:00,  3.15it/s]


In [ ]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()



<ipython-input-12-50e684df3397>:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="refine",
    retriever=retriever
)

In [ ]:
query = "What was the early life and family background of Quaid-e-Azam Muhammad Ali Jinnah?"
response = qa_chain.run(query)
print(response)

Muhammad Ali Jinnah was born in Karachi, likely in 1876, to Jinnahbhai Poonja and Mithibai.  His family were Khoja Shia Muslims from Gujarat, originating from a wealthy merchant background. His father, Jinnahbhai, was a merchant himself, born to a family of textile weavers in Paneli, Gondal state.  His mother hailed from the nearby village of Dhaffa. The family moved to Karachi in 1875, a city experiencing economic growth due to the recently opened Suez Canal, making it a more significant port. Jinnah was the second child in the family.

